In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA

# !pip install openpyxl

combined = pd.read_csv("Combined College Results.csv")
combined.head()

,Institution Name,City,State Abbreviation,Sector Name,Sector,MSI Status,MSI Type,HBCU,PBI,ANNHI,...,Median Earnings of Students Working and Not Enrolled 10 Years After Entry,Native American Non-Tribal Institution (NANTI),Predominantly Black Institution (PBI),Percent of Full-Time Faculty,Percent of Part-Time Undergraduates,Tribal College or University (TCU),Public Institution GASB Endowment Assets,Private not-for-profit or Public Institution FASB Endowment Assets,Number of Undergraduates Age 25 and Older,Percent of Undergraduates Age 25 and Older
0,Modern Technology School,Fountain Valley,CA,"Private for-profit, 2-year",6,1.0,NaN,0.0,0.0,0.0,...,27937.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,62.0,0.186186
1,Modern Technology School,Fountain Valley,CA,"Private for-profit, 2-year",6,1.0,NaN,0.0,0.0,0.0,...,27937.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,62.0,0.186186
2,Modern Technology School,Fountain Valley,CA,"Private for-profit, 2-year",6,1.0,NaN,0.0,0.0,0.0,...,27937.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,62.0,0.186186
3,Modern Technology School,Fountain Valley,CA,"Private for-profit, 2-year",6,1.0,NaN,0.0,0.0,0.0,...,27937.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,62.0,0.186186
4,Modern Technology School,Fountain Valley,CA,"Private for-profit, 2-year",6,1.0,NaN,0.0,0.0,0.0,...,27937.0,0.0,0.0,NaN,0.0,0.0,NaN,NaN,62.0,0.186186


In [4]:
numerical_columns = combined.select_dtypes(include=['int', 'float'])

In [7]:
def hierarchical_fill(df, column, group_hierarchies):
    result = df[column].copy()
    for groups in group_hierarchies:
        still_missing = result.isna()
        if still_missing.sum() == 0:
            break
        group_medians = df.groupby(groups)[column].transform('median')
        result[still_missing] = group_medians[still_missing]
    if result.isna().sum() > 0:
        result.fillna(df[column].median(), inplace=True)
    
    return result

for col in list(df):
    combined[col] = hierarchical_fill(df,
    col,
    [
        ['State', 'Sector Name', 'Institution Size Category Name'],
        ['State', 'Sector Name'],
        ['Sector', 'Sector Name'],
        ['Sector']
    ]
    )



In [13]:
percent_columns = numerical_columns.columns[numerical_columns.columns.str.contains('Percent', case=False)]

In [16]:
for col in list(percent_columns):
    combined[col] = hierarchical_fill(combined,
    col,
    [
        ['State', 'Sector Name', 'Institution Size Category Name'],
        ['State', 'Sector Name'],
        ['Sector', 'Sector Name'],
        ['Sector']
    ]
    )

In [19]:
combined.isnull().sum().sum()/combined.size

0.1508588339551218

In [22]:
number_columns = numerical_columns.columns[numerical_columns.columns.str.contains('Number', case=False)]

In [25]:
for col in list(number_columns):
    combined[col] = hierarchical_fill(combined,
    col,
    [
        ['State', 'Sector Name', 'Institution Size Category Name'],
        ['State', 'Sector Name'],
        ['Sector', 'Sector Name'],
        ['Sector']
    ]
    )

In [28]:
combined.isnull().sum().sum()/combined.size

0.08396989157905102

In [31]:
null_columns = combined[combined.columns[combined.isnull().any()]].isnull().sum()
null_columns = null_columns.sort_values(ascending=False)
print(null_columns)



Hispanic-serving Institution (HSI)                                                              20540
Bachelor's Degree Graduation Rate Within 6 Years - Native Hawaiian or Other Pacific Islander    17085
MSI Type                                                                                        16873
Private not-for-profit or Public Institution FASB Endowment Assets                              15429
Endowment Assets FASB per Student                                                               15369
                                                                                                ...  
Income Earned from Working 10 Hours a Week at State's Minimum Wage                                  9
State Minimum Wage                                                                                  9
Annual Income Estimate                                                                              9
Average Work Study Award                                                          

In [34]:
rate_columns = numerical_columns.columns[numerical_columns.columns.str.contains('Rate', case=False)]

In [37]:
for col in list(rate_columns):
    combined[col] = hierarchical_fill(combined,
    col,
    [
        ['State', 'Sector Name', 'Institution Size Category Name'],
        ['State', 'Sector Name'],
        ['Sector', 'Sector Name'],
        ['Sector']
    ]
    )

In [40]:
combined.isnull().sum().sum()/combined.size

0.052530733856292096

In [46]:
cost_columns = numerical_columns.columns[numerical_columns.columns.str.contains('Attendance', case=False)]

In [49]:
for col in list(cost_columns):
    combined[col] = hierarchical_fill(combined,
    col,
    [
        ['State', 'Sector Name', 'Institution Size Category Name'],
        ['State', 'Sector Name'],
        ['Sector', 'Sector Name'],
        ['Sector']
    ]
    )

In [52]:
combined.isnull().sum().sum()/combined.size

0.04309194811257107

In [55]:
null_columns = combined[combined.columns[combined.isnull().any()]].isnull().sum()
null_columns = null_columns.sort_values(ascending=False)
null_columns.index

Index(['Hispanic-serving Institution (HSI)', 'MSI Type',
       'Private not-for-profit or Public Institution FASB Endowment Assets',
       'Endowment Assets FASB per Student',
       'Public Institution GASB Endowment Assets',
       'Endowment Assets GASB per Student',
       'Instructional Expenses FASB per FTE',
       'Instructional Expenses GASB per Student', 'Bachelor's Degrees Awarded',
       'Average Net Price After Grants- Students Awarded a Grant or Scholarship, 2020-21',
       'Instructional Expenses Per FTE',
       'Cost of Attendance: In State, On Campus',
       'Average In-State Tuition for First-Time, Full-Time Undergraduates',
       'Out-of-State Average Tuition for First-Time, Full-Time Undergraduates',
       'Average Amount of Institutional Grant Aid Awarded to First-Time, Full-Time Undergraduates',
       'Median Earnings of Independent Students Working and Not Enrolled 10 Years After Entry',
       'Median Earnings of Dependent Students Working and Not Enrol

In [61]:
combined.columns

Index(['Institution Name', 'City', 'State Abbreviation', 'Sector Name',
       'Sector', 'MSI Status', 'MSI Type', 'HBCU', 'PBI', 'ANNHI',
       ...
       'Median Earnings of Students Working and Not Enrolled 10 Years After Entry',
       'Native American Non-Tribal Institution (NANTI)',
       'Predominantly Black Institution (PBI)', 'Percent of Full-Time Faculty',
       'Percent of Part-Time Undergraduates',
       'Tribal College or University (TCU)',
       'Public Institution GASB Endowment Assets',
       'Private not-for-profit or Public Institution FASB Endowment Assets',
       'Number of Undergraduates Age 25 and Older',
       'Percent of Undergraduates Age 25 and Older'],
      dtype='object', length=267)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=172841fe-d13b-4ca9-99a4-5716bfc8b95a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>